In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [51]:
dates = pd.date_range(start="2012-01-01",end="2022-03-25").to_frame()
dates2 = pd.date_range(start="2012-01-01",end="2023-03-31").to_frame()

In [52]:
def mort_clean_for_king(data):
    mort = pd.read_csv(data)
    mort.Year_Month = pd.to_datetime(mort.Year_Month)
    mort.rename(columns = {'30-Fixed': '30FixedMortgage'}, inplace = True)
    mort.iloc[:, :-1].to_csv('mortgage_for_king.csv', index = False)
    return (mort)

In [53]:
def mort_clean_for_king_and_aus(data):
    mort = mort_clean_for_king(data)
    mort['30FixedMortgage_3MoMedian'] = mort['30FixedMortgage'].rolling(3).median()[2:]
    mort['30FixedMortgage_3MoMedian'] = mort['30FixedMortgage_3MoMedian'].shift(-2)
    mort.loc[mort.index % 3 == 0, ['Year_Month', '30FixedMortgage_3MoMedian']].to_csv('mortgage_for_austin.csv', index = False)
    return (mort)

In [54]:
mort = mort_clean_for_king_and_aus('mortgage_data_v0404.csv')

In [57]:
def mort_clean_for_king_fulldate(data):
    df_mort = pd.merge(mort, dates2, how = "right", left_on = 'Year_Month', right_on = 0)
    df_mort.sort_values(by = ['Year_Month'], inplace = True)
    df_mort = df_mort.iloc[:, [0, 1]]
    df_mort['30FixedMortgage'] = df_mort['30FixedMortgage'].ffill()
    df_mort.to_csv('mortgage_for_king.csv', index = False)

In [58]:
mort_clean_for_king_fulldate(mort)

In [67]:
def stock_clean_for_king(data, stock):
    df = pd.read_csv(data)
    df.Date = pd.to_datetime(df.Date)
    df_new = pd.merge(df, dates, how = "right", left_on = 'Date', right_on = 0)
    df_new.sort_values(by = ['Date'], inplace = True)
    df_new.reset_index(inplace = True)
    df_new = df_new.iloc[:, [1, 5]]
    df_new['Close'] = df_new['Close'].ffill()
    df_new['Close'] = df_new['Close'].bfill()
    df_new.to_csv('stock_{}_for_king.csv'.format(stock), index = False)

In [68]:
def stock_clean_for_aus(data, stock):
    df = pd.read_csv(data)
    df.Date = pd.to_datetime(df.Date)
    df = df.groupby(df['Date'].dt.to_period('Q'))['Close'].median().to_frame()
    mort_date = mort.loc[mort.index % 3 == 0, ['Year_Month']]
    df['Date_3Mo'] = mort_date.iloc[:-4, :]['Year_Month'].to_list()
    df.reset_index(inplace = True)
    df.loc[:, ['Date_3Mo', 'Close']].to_csv('stock_{}_for_austin.csv'.format(stock), index = False)

In [69]:
def stock_clean(stock):
    stock_clean_for_king('{}_0328.csv'.format(stock), stock)
    stock_clean_for_aus('{}_0328.csv'.format(stock), stock)

In [70]:
for stock in ['SPY', 'QQQ', 'DGL']:
    stock_clean(stock)